# **Imports**

In [1]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import json
import re
import PyPDF2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dask import dataframe as dd
import plotly.express as px
import math
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from src.text_cleaning import *
import ast
from numpy.linalg import norm
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from src.weights import *
from nltk.cluster.util import cosine_distance
from nltk.cluster import KMeansClusterer
import nltk
import random
import pacmap
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")
from src.proj_functs import *
from src.projections import *

Cosine Similarity b1-b2: -0.39971082824723836
Cosine Similarity b1-b2: -0.3283980118771736
Cosine Similarity b1-b2: -0.4343392279125446
Cosine Similarity b1-b2: -0.3556753325971033
Cosine Similarity b1-b2: -0.25868816236356595
Cosine Similarity b1-b2: -0.3282046873655816
Cosine Similarity b1-b2: -0.34722643033966344
Cosine Similarity b1-b2: -0.35393206801784416
Cosine Similarity b1-b2: -0.3328882761888091
Cosine Similarity b1-b2: -0.3341682316005257
Cosine Similarity b1-b2: -0.02389130901932727
Cosine Similarity b1-b2: -0.16105655829265958
Cosine Similarity b1-b2: -0.0879007369208663
Cosine Similarity b1-b2: 0.09162228538266397


# **Computations**

In [2]:
def read(text) :
    return text.replace('_', ' ')

def representative_speeches(df, nbr : int):

    biggest_on_axis_1 = df.sort_values(by=['cos axe 1'], ascending = False).head(nbr)
    lowest_on_axis_1 = df.sort_values(by=['cos axe 1'], ascending = True).head(nbr)
    biggest_on_axis_2 = df.sort_values(by=['cos axe 2'], ascending = False).head(nbr)
    lowest_on_axis_2 = df.sort_values(by=['cos axe 2'], ascending = True).head(nbr)

    biggest_on_axis_1['text'] = biggest_on_axis_1['text'].apply(read)
    lowest_on_axis_1['text'] = lowest_on_axis_1['text'].apply(read)
    biggest_on_axis_2['text'] = biggest_on_axis_2['text'].apply(read)
    lowest_on_axis_2['text'] = lowest_on_axis_2['text'].apply(read)

    return (biggest_on_axis_1, lowest_on_axis_1, biggest_on_axis_2, lowest_on_axis_2)

def scatter_representative_speeches(df, nbr:str) :

    a = representative_speeches(df, nbr)[0]
    b = representative_speeches(df, nbr)[1]
    c = representative_speeches(df, nbr)[2]
    d = representative_speeches(df, nbr)[3]

    a['color'] = 1
    b['color'] = 2
    c['color'] = 3
    d['color'] = 4

    df_join = pd.concat([a, b, c, d], ignore_index=True)

    fig = px.scatter(df_join, x='cos axe 1', y='cos axe 2', color="color", hover_data=['text'])
    fig.show()

In [3]:
scatter_representative_speeches(df_2017, 30)

In [4]:
representative_speeches(df_2017, 10)[1]

,Unnamed: 0,text,sentence_embedding,source,party,keywords,year,proj_embedding_x,proj_embedding_y,cos axe 1,cos axe 2
7113,7113.0,would appropri consider amount independ oper b...,"['-0.06787063758851469', '-0.10807076212260336...",par,Con,[],2017,-0.493482,0.029099,-0.687230,0.277956
7786,7786.0,contrast debat realli issu aros report thank l...,"['-0.03900079175152582', '0.010084351390706736...",par,Con,[],2017,-0.452862,0.120997,-0.684279,0.388296
21893,21893.0,think must follow execut function northern ire...,"['-0.0607481674626385', '0.08534007643022573',...",par,Con,"['edge', 'tech']",2017,-0.388289,0.092781,-0.681172,0.372362
20919,20919.0,rais excel point also rais european chemic age...,"['-0.06470761626460986', '-0.00350242579236670...",par,Lab,['excel'],2017,-0.402341,0.079714,-0.673885,0.347646
11865,11865.0,come moment sanction enabl continu impos updat...,"['-0.07121734986012887', '0.026574939805798392...",par,Con,[],2017,-0.468107,-0.014290,-0.673627,0.220233
21891,21891.0,take learn point utmost serious alway respect ...,"['-0.16427862983438665', '-0.03334673335832106...",par,Con,[],2017,-0.330935,0.079536,-0.672653,0.368399
21160,21160.0,regul turn briefli respond name nottingham eas...,"['-0.04656238799476917', '0.029661750132907505...",par,Con,[],2017,-0.411626,0.117045,-0.669494,0.388251
6997,6997.0,agre know problem easi solv central today deba...,"['-0.06971022376466604', '-0.07991989402134969...",par,Lab,[],2017,-0.324058,0.094296,-0.666951,0.389965
21154,21154.0,sure follow contribut reliev know make brief r...,"['-0.12890731611804954', '0.029409835315031718...",par,Lab,[],2017,-0.380314,0.085760,-0.662493,0.355496
17824,17824.0,need make progress forgiv much explain procedu...,"['-0.022034905724411302', '0.03625110354263639...",par,Con,['tech'],2017,-0.261493,0.111456,-0.658765,0.446574
